In [589]:
import pandas as pd
from pathlib import Path
import numpy as np

In [594]:

def get_excel_files(path):
    excel_files = []
    for file in Path(path).glob('*.xls*'):
        excel_files.append(file)
        # logger.info(f"Found file: {file}")
    #return list if it has values else end program
    if excel_files:
        return excel_files
    else:
        raise Exception("No excel files found")


    

def add_iso_date(file : Path):

    dt_df = pd.read_excel(file,nrows=1,header=None)
    dt = pd.to_datetime(dt_df[0].str.split('Date :',expand=True)[1]).dt.strftime('%Y-%m-%d_%H-%M-%S')

    file_name = f"{dt}_{file.stem}{file.suffix}"
    file.rename(file.parent.parent.joinpath('processed',file_name))
    # logger.info(f"Renamed file: {file.stem} -> {file_name}")
    return file_name

In [633]:
files = get_excel_files('../files/raw/unprocessed')

In [634]:
df = pd.read_excel(files[0],engine='xlrd',nrows=5000,header=None)

In [637]:
df[df[0].str.contains('Year')==True].index[1] - 5 #the first company names is usually 4-5 rows before the columns.


388

393

In [625]:
pd.to_datetime(df[0].str.split('Date :',expand=True)[1]).dt.strftime('%Y-%m-%d_%H-%M-%S')[0]

'2021-12-14_15-24-06'

In [582]:
# cleaned_frames = {}

# for index, dataframe in dfs.items():
#     # dataframe = dfs[1].copy()
#     df_new = dataframe.copy()
#     df_new = df_new.dropna(subset=[0],how='all')
#     unit_vals = df_new.iloc[0].str.split('\n',expand=True)
#     units = unit_vals[unit_vals.columns[-1]].str.replace('\(|\)','',regex=True).str.strip()
#     vals = unit_vals[unit_vals.columns[:-1]].fillna('').agg(' '.join,axis=1).str.strip()
#     df_new.columns = pd.MultiIndex.from_tuples(list(zip(vals,units)))
#     df_new = df_new.iloc[1:]
#     df_new['Year'] = pd.to_datetime(df_new.iloc[:,0],errors='coerce').dt.strftime('%d/%m/%Y')
#     df_new = df_new.dropna(subset=df_new.iloc[:,:1].columns)
#     case = df_new.iloc[:,-1].unique()[0]
#     df3 = df_new.set_index('Year').stack([0,1]).unstack(0)
#     cols = [pd.to_datetime(x).strftime('%d/%m/%Y')[0] for x in df3.columns]
#     df3 = df3.assign(caseName=case).set_index('caseName',append=True).reset_index().rename(columns={'level_1' : 'ValueType', 'level_0' : 'Units'})
#     df3.columns = ['ValueType','Units','CaseName'] + cols
#     df3 = df3[['CaseName','ValueType','Units'] + cols]
#     cleaned_frames[index] = df3
# final = pd.concat(cleaned_frames)

# k = { x : pd.to_datetime(x) for x in final.iloc[:,3:].columns}
# cols = dict(sorted(k.items(), key=lambda item: item[1])).keys()

# final = final[['CaseName','ValueType','Units'] + list(cols)]

# sort_col = {
#     "1": "Oil Net",
#     "2": "Gas Net",
#     "3": "NGL Net",
#     "4": "Oil Price",
#     "5": "Gas  Price",
#     "6": "NGL  Price",
#     "8": "Oil  & Gas Rev. Net",
#     "9": "Misc.  Rev. Net",
#     "7": "Costs Net",
#     "10": "Taxes Net",
# "11": "Gas Gross",
# "12": "NGL Gross",
# "13": "Oil Gross",
# "14": "Invest. Net",
# "15": "NonDisc. CF Annual",
# "16": "Cum Disc.CF"
# }

# final = final.fillna(0)
# final['ValueType'] = pd.Categorical(final['ValueType'],sort_col.values())
# final.index.names = ['idx','index_row']
# final = final.sort_values(['idx','ValueType'])

In [588]:
# final.to_excel(f"../files/processed/{pd.Timestamp('now').strftime('%Y-%m-%d')}_oildata.xlsx",index=False)

df2